In [4]:
# 导入相关库
import pandas as pd
from pandas import DataFrame,Series
from pandas.io.excel import ExcelWriter

In [5]:
# 定义参数
dx = 200
dy = 100
x_min = 400
x_max = 2400
y_min = 0
y_max = 1000
inputFileName = u'data/重石中沙轻煤跑车合.xls'
outputFileName = u'data/重石中沙轻煤跑车合(dx=200,dx=100).xls'
sheets = ['highstone','middlesand','lowcoal','running']
x_column = u'发动机转速-rpm'
y_column = u'扭矩-N.m'
columns = [x_column,y_column]

In [6]:
# 定义数据转化函数
def transform(data):
    x = data[x_column].apply(lambda x:trunc(x/dx)*dx)
    y = data[y_column].apply(lambda x:round(x/dy)*dy)
    x = Series(list(x))
    y = Series(list(y))
    df = DataFrame({'x':x,'y':y,'c':[1]*len(x)})
    x_all = range(x_min,x_max+dx,dx)
    df_x_all = DataFrame({'x':x_all,'y':[y_min]*len(x_all),'c':[0]*len(x_all)})
    y_all = range(y_min,y_max+dy,dy)
    df_y_all = DataFrame({'x':[x_min]*len(y_all),'y':y_all,'c':[0]*len(y_all)})
    df = pd.concat([df,df_x_all,df_y_all])
    df = df.groupby(['x','y'])['c'].sum()
    df = df.fillna(0)
    df = df.unstack()
    df = df.fillna(0)
    df = df.swapaxes(0,1)
    df = df.sort_index(ascending=False)
    return df

In [8]:
# 生成数据
writer = ExcelWriter(outputFileName)

# 转化每一个sheet数据
dataList = []
for i,sheet in enumerate(sheets):
    data = pd.io.excel.read_excel(inputFileName,sheet)[columns]
    df = transform(data)
    df.to_excel(writer,sheet)
    dataList.append(data)

# 生成汇总数据
dataAll = pd.concat(dataList)
df = transform(dataAll)
df.to_excel(writer,'all')

# 保存
writer.save()